<a href="https://colab.research.google.com/github/karsarobert/Deep-Learning-2023/blob/main/11/PTE_DL11_Senteces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning gyakorlat


## 11. gyakorlat: word2vec
### 2022. november 23.

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Szöveg előkészítése a TensorFlow modellekkel való használatra


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l09c02_nlp_padding.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l09c02_nlp_padding.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

A gépi tanulási modellben felhasználható szöveg előkészítésének magas szintű lépései a következők:

1.   Tokenizáljuk a szavakat, hogy számértékeket kapjunk számukra.
2.   A mondatok számsorozatainak létrehozása
3.   Állítsuk be a sorozatok hosszát úgy, hogy mindegyik azonos legyen.


## Importáljuk a szükséges osztályokat

In [ ]:
# Import Tokenizer and pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Írjunk néhány mondatot

Lehet változtatni is a szövegen.


In [ ]:
sentences = [
    'My favorite food is ice cream',
    'do you like ice cream too?',
    'My dog likes ice cream!',
    "your favorite flavor of icecream is chocolate",
    "chocolate isn't good for dogs",
    "your dog, your cat, and your parrot prefer broccoli"
]
print(sentences)


['My favorite food is ice cream', 'do you like ice cream too?', 'My dog likes ice cream!', 'your favorite flavor of icecream is chocolate', "chocolate isn't good for dogs", 'your dog, your cat, and your parrot prefer broccoli']


## Hozzuk létre a tokenizálót és definiáljunk egy szókészleten kívüli tokent
A Tokenizer létrehozásakor megadhatja a szótárban lévő szavak maximális számát. Megadhat egy tokent is, amely a szókészleten kívüli (OOV), azaz a szótárban nem szereplő szavakat jelöli. Ez az OOV token akkor kerül felhasználásra, amikor olyan mondatokhoz készít szekvenciákat, amelyek olyan szavakat tartalmaznak, amelyek nem szerepelnek a szóindexben.

In [ ]:
tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")

## A szavak tokenizálása

In [ ]:
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'your': 2, 'ice': 3, 'cream': 4, 'my': 5, 'favorite': 6, 'is': 7, 'dog': 8, 'chocolate': 9, 'food': 10, 'do': 11, 'you': 12, 'like': 13, 'too': 14, 'likes': 15, 'flavor': 16, 'of': 17, 'icecream': 18, "isn't": 19, 'good': 20, 'for': 21, 'dogs': 22, 'cat': 23, 'and': 24, 'parrot': 25, 'prefer': 26, 'broccoli': 27}


## Alakítsuk a mondatokat szekvenciákká

Mostantól minden szónak egyedi száma van a szóindexben.  A mondatban lévő szavak azonban meghatározott sorrendben vannak. Nem lehet csak úgy véletlenszerűen összekeverni a szavakat, és az eredmény egy mondat lesz.

A következő lépés tehát a szöveg olyan módon történő ábrázolásához, amelyet a gépi tanulási programok értelmesen használhatnak, az, hogy a szövegben szereplő mondatokat ábrázoló számsorozatokat hozzunk létre.

Minden egyes mondatot olyan szekvenciává alakítunk át, amelyben minden egyes szót a szóindexben szereplő számmal helyettesítünk.

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
print (sequences)

[[5, 6, 10, 7, 3, 4], [11, 12, 13, 3, 4, 14], [5, 8, 15, 3, 4], [2, 6, 16, 17, 18, 7, 9], [9, 19, 20, 21, 22], [2, 8, 2, 23, 24, 2, 25, 26, 27]]


## A szekvenciák mindegyike legyen azonos hosszúságú.

Később, amikor a szekvenciákat egy neurális hálózatba tápláljuk a modell betanításához, a szekvenciáknak egyforma méretűeknek kell lenniük. Jelenleg a szekvenciák különböző hosszúságúak, ezért a következő lépés az, hogy a szekvenciák azonos méretűek legyenek, vagy nullákkal való kitöltéssel és/vagy csonkítással.

Az f.keras.preprocessing.sequence.pad_sequences segítségével nullákat adhatunk hozzá a szekvenciákhoz, hogy mindegyik azonos hosszúságú legyen. Alapértelmezés szerint a kitöltés a szekvenciák elejére kerül, de megadhatja, hogy a szekvenciák végére kerüljön a kitöltés.

Opcionálisan megadhatja a maximális hosszúságot, ameddig a szekvenciákat fel kell tölteni. A megadott maximális hossznál hosszabb szekvenciákat a rendszer le fogja vágni. Alapértelmezés szerint a szekvenciákat a szekvencia elejétől kezdve vágja le a rendszer, de megadhatja, hogy a szekvencia végétől kezdve vágja le.

Ha nem adja meg a maximális hosszúságot, akkor a szekvenciák a leghosszabb mondat hosszának megfelelően lesznek kitöltve.

A szekvenciák kitöltésére és csonkítására vonatkozó összes lehetőséget lásd a https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences oldalon.



In [ ]:
padded = pad_sequences(sequences)
print("\nWord Index = " , word_index)
print("\nSequences = " , sequences)
print("\nPadded Sequences:")
print(padded)



Word Index =  {'<OOV>': 1, 'your': 2, 'ice': 3, 'cream': 4, 'my': 5, 'favorite': 6, 'is': 7, 'dog': 8, 'chocolate': 9, 'food': 10, 'do': 11, 'you': 12, 'like': 13, 'too': 14, 'likes': 15, 'flavor': 16, 'of': 17, 'icecream': 18, "isn't": 19, 'good': 20, 'for': 21, 'dogs': 22, 'cat': 23, 'and': 24, 'parrot': 25, 'prefer': 26, 'broccoli': 27}

Sequences =  [[5, 6, 10, 7, 3, 4], [11, 12, 13, 3, 4, 14], [5, 8, 15, 3, 4], [2, 6, 16, 17, 18, 7, 9], [9, 19, 20, 21, 22], [2, 8, 2, 23, 24, 2, 25, 26, 27]]

Padded Sequences:
[[ 0  0  0  5  6 10  7  3  4]
 [ 0  0  0 11 12 13  3  4 14]
 [ 0  0  0  0  5  8 15  3  4]
 [ 0  0  2  6 16 17 18  7  9]
 [ 0  0  0  0  9 19 20 21 22]
 [ 2  8  2 23 24  2 25 26 27]]


In [ ]:
# Specify a max length for the padded sequences
padded = pad_sequences(sequences, maxlen=15)
print(padded)

[[ 0  0  0  0  0  0  0  0  0  5  6 10  7  3  4]
 [ 0  0  0  0  0  0  0  0  0 11 12 13  3  4 14]
 [ 0  0  0  0  0  0  0  0  0  0  5  8 15  3  4]
 [ 0  0  0  0  0  0  0  0  2  6 16 17 18  7  9]
 [ 0  0  0  0  0  0  0  0  0  0  9 19 20 21 22]
 [ 0  0  0  0  0  0  2  8  2 23 24  2 25 26 27]]


In [ ]:
# Put the padding at the end of the sequences
padded = pad_sequences(sequences, maxlen=15, padding="post")
print(padded)

[[ 5  6 10  7  3  4  0  0  0  0  0  0  0  0  0]
 [11 12 13  3  4 14  0  0  0  0  0  0  0  0  0]
 [ 5  8 15  3  4  0  0  0  0  0  0  0  0  0  0]
 [ 2  6 16 17 18  7  9  0  0  0  0  0  0  0  0]
 [ 9 19 20 21 22  0  0  0  0  0  0  0  0  0  0]
 [ 2  8  2 23 24  2 25 26 27  0  0  0  0  0  0]]


In [ ]:
# Limit the length of the sequences, you will see some sequences get truncated
padded = pad_sequences(sequences, maxlen=3)
print(padded)

[[ 7  3  4]
 [ 3  4 14]
 [15  3  4]
 [18  7  9]
 [20 21 22]
 [25 26 27]]


## Mi történik, ha néhány mondat olyan szavakat tartalmaz, amelyek nem szerepelnek a szóindexben?

Itt használjuk a "szókészleten kívüli" tokent. Próbáljon meg szekvenciákat generálni néhány olyan mondathoz, amelyekben olyan szavak vannak, amelyek nem szerepelnek a szóindexben.

In [ ]:
# Try turning sentences that contain words that
# aren't in the word index into sequences.
# Add your own sentences to the test_data
test_data = [
    "my best friend's favorite ice cream flavor is strawberry",
    "my dog's best friend is a manatee"
]
print (test_data)

# Remind ourselves which number corresponds to the
# out of vocabulary token in the word index
print("<OOV> has the number", word_index['<OOV>'], "in the word index.")

# Convert the test sentences to sequences
test_seq = tokenizer.texts_to_sequences(test_data)
print("\nTest Sequence = ", test_seq)

# Pad the new sequences
padded = pad_sequences(test_seq, maxlen=10)
print("\nPadded Test Sequence: ")

# Notice that "1" appears in the sequence wherever there's a word
# that's not in the word index
print(padded)

["my best friend's favorite ice cream flavor is strawberry", "my dog's best friend is a manatee"]
<OOV> has the number 1 in the word index.

Test Sequence =  [[5, 1, 1, 6, 3, 4, 16, 7, 1], [5, 1, 1, 1, 7, 1, 1]]

Padded Test Sequence: 
[[ 0  5  1  1  6  3  4 16  7  1]
 [ 0  0  0  5  1  1  1  7  1  1]]
